In [70]:
import sys
import os 

In [71]:
sys.path.insert(1, os.path.abspath( os.path.dirname("../")) )
sys.path.insert(1, os.path.abspath( os.path.dirname("../sql")) )

In [ ]:
import langchain
import streamlit as st
from agents.agent_parser.find_compagny_type_parser import (
    society_type_parser,
    SocietyTypeParser,
)
from langchain_core.prompts import PromptTemplate
from langchain_mistralai import ChatMistralAI
from utils.session_state_management import save_user_infos_in_session_state

import os
from dotenv import load_dotenv
import sqlite3

load_dotenv()

True

In [73]:
from utils.user_infos_form_state import get_user_information

In [74]:

conn = sqlite3.connect("../Prospection.db")


In [75]:

cursor = conn.cursor()
cursor.execute("""SELECT 
    id,
    firstname, 
    lastname, 
    email,
    compagny_type,
    activity,
    activity_large_description
FROM User_infos
WHERE id = ?""", (1,))
row = cursor.fetchall()
data = [element for elements in row for element in elements]
user_infos = {
    "id": data[0],
    "firstname": data[1],
    "lastname": data[2],
    "email": data[3],
    "compagny_type": data[4],
    "compagny_activity": data[5],
    "compagny_large_activity": data[6]}

In [76]:
user_infos

{'id': 1,
 'firstname': 'Louis',
 'lastname': 'Martin du Nord',
 'email': 'l.martindunord@gmail.com',
 'compagny_type': 'Freelance',
 'compagny_activity': 'test',
 'compagny_large_activity': 'test'}

In [77]:
prompt = """Je suis {work_description}  looking to identify my ideal client peux tu me proposer
        une liste de type d'entreprise (secteur, taille et model économique) qui aurait un interet strategique à faire appel à mes services.
        Puis pour chaque type d'entreprise, indique moi le ou les postes décisionnaire pour ce type d'entreprise.
        je cible principalement des entreprises francophones.
        
        voici le format attendue pour la réponse :{society_type_parser} ne répond uniquement en utilisant ce format. ne dit rien de plus
    """

In [78]:
summary_prompt_template = PromptTemplate(
       input_variables=["work_description"],
       template=prompt,
       partial_variables={
           "format_instructions": SocietyTypeParser.get_format_instructions()
       })

AttributeError: get_format_instructions

In [ ]:
os.environ["MISTRAL_API_KEY"] = os.getenv("MISTRAL_API_KEY")
print(os.environ["MISTRAL_API_KEY"])

jFwjuDFmy9adO3RpRTQ5AGVNmnj5KORm


In [ ]:
llm = llm = ChatMistralAI(
        model="mistral-large-latest",
        temperature=0,
        max_retries=2,
    )

In [ ]:
actity_infos = user_infos.get("compagny_activity")
print(actity_infos)

test


In [ ]:
chain = summary_prompt_template | llm | society_type_parser
res: SocietyTypeParser = chain.invoke(input={"work_description": actity_infos,"society_type_parser":SocietyTypeParser})

OutputParserException: Invalid json output: ```python
from pydantic import BaseModel
from typing import List, Optional

class SocietyTypeParser(BaseModel):
    secteur: str
    taille: str
    modele_economique: str
    interet_strategique: str
    postes_decisionnaires: List[str]

# Exemple de données
society_types = [
    SocietyTypeParser(
        secteur="Technologie de l'information",
        taille="PME",
        modele_economique="SaaS",
        interet_strategique="Optimisation des processus internes et amélioration de l'expérience utilisateur",
        postes_decisionnaires=["Directeur Technique (CTO)", "Directeur des Systèmes d'Information (DSI)"]
    ),
    SocietyTypeParser(
        secteur="E-commerce",
        taille="Grande entreprise",
        modele_economique="B2C",
        interet_strategique="Amélioration de la conversion et de la rétention des clients",
        postes_decisionnaires=["Directeur Marketing", "Directeur de l'Expérience Client"]
    ),
    SocietyTypeP

In [ ]:
print(res)